In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
query=input("title?")
if " " in query:
    query=query.replace(" ","-")
driver = webdriver.Chrome()

In [ ]:
driver.get('https://timesofindia.indiatimes.com/topic//news')

html_doc = driver.page_source

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
div = soup.find('div', class_= 'pagination')

In [ ]:

list_of_page_links=[]
for a in div.find_all('a'):
    try:
        s=a['href']
        s='https://timesofindia.indiatimes.com'+s
        print(s)
        list_of_page_links.append(s)
    except:
        continue

In [ ]:
list_of_page_links=list_of_page_links[:8]

In [ ]:
list_of_page_links

In [ ]:
with open('list_of_page_links.txt', 'w') as f:
    for item in list_of_page_links:
        f.write("%s\n" % item)

In [ ]:

list_of_links=[]

with open('list_of_page_links.txt', 'r') as f:
      for line in f:
        #We used Chrome Drive in our Case , We can even use PhantomJS 
        #the line refers to page link
        driver = webdriver.Chrome()
        driver.get(line)

        html_doc = driver.page_source
        
        soup = BeautifulSoup(driver.page_source, 'lxml')
        div = soup.find('div', class_= 'tab_content')

        #Since cms links are in a tag.
        for a in div.find_all('a'):
        
            try:
                s=a['href']
        
                u=s[::-1]
                k=u[:3]
        #checking if end is cms
                if k == 'smc':
                    s='https://timesofindia.indiatimes.com'+s
                    list_of_links.append(s)
            except:
                    continue

In [ ]:
print(len(list_of_links))

In [ ]:
with open('list_of_links22.txt', 'w') as f:
    for item in list_of_links:
        f.write("%s\n" % item)

In [ ]:
import newspaper
from newspaper import Article 

In [ ]:
article_title=[]
article_text=[]
article_summary=[]
article_keywords=[]

with open('list_of_links22.txt', 'r') as f:
      for url in f:
            try:
                url = url.replace("\n", "")
                toi_article = Article(url, language="en")
                toi_article.download() 
  
                #To parse the article 
                toi_article.parse() 
  
                #To perform natural language processing ie..nlp 
                toi_article.nlp() 
        
                #To extract title 
                article_title.append(toi_article.title) 
                
                #To extract text 
                article_text.append(toi_article.text) 
                
                #To extract summary 
                article_summary.append(toi_article.summary) 
                
                #To extract keywords 
                article_keywords.append(toi_article.keywords) 
            
                print("****************************************************************")
            
            except:
                continue

In [ ]:
print(len(article_title))
print(len(article_text))
print(len(article_summary))
print(len(article_keywords))

In [ ]:
d = {'Title':article_title,'Text':article_text,'Summary':article_summary,'Keywords':article_keywords}

In [ ]:
import pandas as pd
df=pd.DataFrame(d)

In [ ]:
df.head(10)

In [ ]:
df.to_csv('reliance industries.csv', sep=',')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline

In [ ]:
df=pd.read_csv('reliance industries.csv')
df.shape[1] 

In [ ]:
urls=[]
with open('list_of_links22.txt', 'r') as f:
      for url in f:
            print(url)
            urls.append(url)

In [ ]:
urls=[]
with open('list_of_links22.txt', 'r') as f:
      for url in f:
            print(url)
            urls.append(url)

In [ ]:
df

In [ ]:
main_title = list(df['Title'])

In [ ]:
df["main_title"]=main_title

In [ ]:
import re
import string
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#+_-]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(row):
    """
        text: a string
        return: modified initial string
    """
    text=row['Summary']
    # 1. Lowercase text
    text = text.lower() 
    # 2. Replace '/', '()', '{}', '[]', '|', '@', ',', ';' by space
    text = REPLACE_BY_SPACE_RE.sub(' ',text)
    # 3. Delete the non-alphanumerics
    text = BAD_SYMBOLS_RE.sub('',text)
    # 4. Delete stopwords
    text_words = text.split()
    text_words = [t for t in text_words if t not in STOPWORDS]
    text = ' '.join(text_words)
    return text


In [ ]:
df['Title'] = df.apply (lambda row: text_prepare(row),axis=1)

In [ ]:
df['Text'] = df.apply (lambda row: text_prepare(row),axis=1)

In [ ]:
df['Summary'] = df.apply (lambda row: text_prepare(row),axis=1)

In [ ]:
df.head(5)

In [ ]:
def text_prepare_kw(row):
    text=row['Keywords']
    text=text[1:len(text)-1]
    import re
    text=re.sub("[\']",'',text)
    text=text.split(',')
    text = ' '.join(text)
    return text

In [ ]:
df['Keywords']=df.apply (lambda row: text_prepare_kw(row),axis=1)

In [ ]:
df['Tit_sum_key']=df['Title']+df['Summary']+df['Keywords']

In [ ]:
df['Tit_sum_key']

In [ ]:
df.head(10)

In [ ]:
b=["fraud","litigation","case","default","prosecute","sue","bounce","withhold","dishonoured","non compliance","unlawful","forensic","audit","violation","fleed","fugitive","disqualification","FIR","negative","pledge","adverse","hidden","undisclosed","invocation","encumbrance","revoke"]
answers = []
data = list(df['Tit_sum_key'])

for i in data:
    flag_check = 'positive'
    for j in b:
        if j in i:
            flag_check = 'negative'
    answers.append(flag_check)

In [ ]:
print(len(answers))

In [ ]:
df['sentiment'] = answers

In [ ]:
df.to_csv('updated_work.csv',index = False)

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df_x = df[['main_title','url','sentiment']]

In [ ]:
df_x.to_csv('final.csv')

In [ ]:
df_y=df[df['sentiment']=="negative"]

In [ ]:
df_negative=df_y[['main_title','url','sentiment']]